# **This Notebook gives an idea of how to explore a massive dataset. This is just a sample. Follow git repo instructions for Lab 07**

## Feel free to reach out if you have doubts
## vzm5268@psu.edu
## Office Hours: Wednesday 10-12PM

First, import the `datascience` library and some other needed resources, as we've done many times in the past.

In [36]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import math
from scipy import stats
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=np.VisibleDeprecationWarning)

Following the same procedure as in [Lab 6](https://github.com/DS200-SP2024-Hunter/Week06-DueFeb23), mount the google drive:

In [37]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Then `cd` (change directory) to the correct path where the giant dataset is stored. (Your own code may look slightly different; use the same code you used in [Lab 6](https://github.com/DS200-SP2024-Hunter/Week06-DueFeb23).)

In [38]:
cd /content/drive/MyDrive/Colab\ Notebooks/data

/content/drive/MyDrive/Colab Notebooks/data


Read in the massive `flights.csv` dataset (massive).  This takes a while.

In [39]:
flights = Table.read_table('flights.csv')

/usr/local/lib/python3.10/dist-packages/datascience/tables.py:163: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv(filepath_or_buffer, *args, **vargs)


Now read the smaller `airports.csv` and `airlines.csv` datasets:

In [46]:
airports = Table.read_table('airports.csv')
airlines = Table.read_table('airlines.csv')

Think about what information might be interesting to display on a map.  Here are the variables available in the large dataset:

In [40]:
flights.labels

('YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY')

Similarly, the columns in the other two datasets:

In [41]:
airlines.labels

('IATA_CODE', 'AIRLINE')

In [42]:
airports.labels

('IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE')

We know that the mapping functions from Chapter 8 need columns labeled `Long` and `Lat`, so let's change some of the column names in the `airports` table:

In [47]:
airports.relabel('AIRPORT', 'Airport Name')
airports.relabel('LATITUDE', 'lat')
airports.relabel('LONGITUDE', 'long')

IATA_CODE,Airport Name,CITY,STATE,COUNTRY,lat,long
ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.6524,-75.4404
ABI,Abilene Regional Airport,Abilene,TX,USA,32.4113,-99.6819
ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.0402,-106.609
ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.4491,-98.4218
ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.5355,-84.1945
ACK,Nantucket Memorial Airport,Nantucket,MA,USA,41.2531,-70.0602
ACT,Waco Regional Airport,Waco,TX,USA,31.6113,-97.2305
ACV,Arcata Airport,Arcata/Eureka,CA,USA,40.9781,-124.109
ACY,Atlantic City International Airport,Atlantic City,NJ,USA,39.4576,-74.5772
ADK,Adak Airport,Adak,AK,USA,51.878,-176.646


## Sample Task: Figure out how many flights per airline

Now that all the data are ready to analyze, what follows is a series of tasks that might help you decide what to do for your own map.  You can adapt this code however you want.

Let's figure out how many different airlines are in the big dataset.  This information is contained in the `AIRLINE` column, so let's first select this column (to reduce the computational burden of such a massive dataset) and then `group` by it:

In [48]:
Count_by_airline = flights.select('AIRLINE').group('AIRLINE')
Count_by_airline

AIRLINE,count
AA,725984
AS,172521
B6,267048
DL,875881
EV,571977
F9,90836
HA,76272
MQ,294632
NK,117379
OO,588353


It's hard to understand the two-letter codes, so we can decipher these codes using the information in the `airlines` table.  We'll need to `join` the `Count_by_airline` and `airlines` tables using the columns that list these codes:


In [49]:
Count_by_airline.join('AIRLINE', airlines, 'IATA_CODE')

AIRLINE,count,AIRLINE_2
AA,725984,American Airlines Inc.
AS,172521,Alaska Airlines Inc.
B6,267048,JetBlue Airways
DL,875881,Delta Air Lines Inc.
EV,571977,Atlantic Southeast Airlines
F9,90836,Frontier Airlines Inc.
HA,76272,Hawaiian Airlines Inc.
MQ,294632,American Eagle Airlines Inc.
NK,117379,Spirit Air Lines
OO,588353,Skywest Airlines Inc.


Notice that `join` automatically renamed a column `AIRLINE_2` to prevent two columns with the same name.  Let's sort in decreasing order of number of flights and then relabel a couple of the columns.  

In [50]:
Count_by_airline = Count_by_airline.join('AIRLINE', airlines, 'IATA_CODE')
Count_by_airline = Count_by_airline.sort('count', descending = True)

# Notice that the relabel method changes an existing Table object.  This is different from the relabeled method, which creates a new Table object.
Count_by_airline.relabel('AIRLINE', 'Airline Code')
Count_by_airline.relabel('AIRLINE_2', 'Airline Name')

Count_by_airline.show()

Airline Code,count,Airline Name
WN,1261855,Southwest Airlines Co.
DL,875881,Delta Air Lines Inc.
AA,725984,American Airlines Inc.
OO,588353,Skywest Airlines Inc.
EV,571977,Atlantic Southeast Airlines
UA,515723,United Air Lines Inc.
MQ,294632,American Eagle Airlines Inc.
B6,267048,JetBlue Airways
US,198715,US Airways Inc.
AS,172521,Alaska Airlines Inc.


## Sample Task: Calculate average departure delay for each airline

In the previous code block, we renamed `Count_by_airline` multiple times, once after each step.  Here is a different approach that performs multiple tasks in succession and only names the new object after everything is complete.  The purpose of this code is to calculate the average departure delay for each airline.  Notice that the `group` method can be used with the `nanmean` method (which calculates the mean while ignoring `nan` or not-a-number values) from `numpy`.  

In [51]:
avg_delays = (flights
     .select('AIRLINE', 'DEPARTURE_DELAY')
     .relabeled('AIRLINE', 'Airline Code')
     .group('Airline Code', np.nanmean)
     .relabeled('DEPARTURE_DELAY nanmean', 'Average Delay')
     .join('Airline Code', airlines, 'IATA_CODE')
     .relabeled('AIRLINE', 'Airline Name')
     .sort('Average Delay')
)
# Print the table we just created
avg_delays.show()

Airline Code,Average Delay,Airline Name
HA,0.485713,Hawaiian Airlines Inc.
AS,1.7858,Alaska Airlines Inc.
US,6.14114,US Airways Inc.
DL,7.36925,Delta Air Lines Inc.
OO,7.8011,Skywest Airlines Inc.
EV,8.71593,Atlantic Southeast Airlines
AA,8.90086,American Airlines Inc.
VX,9.0226,Virgin America
MQ,10.1252,American Eagle Airlines Inc.
WN,10.582,Southwest Airlines Co.


## Sample Task: As an illustration, let's consider only flights by American and American Eagle

Since the dataset is so large, let's consider just a subset of the flights when thinking about what to put on a map.  We can pick out only the rows in which the flights are operated by American (AA) or American Eagle (MQ):

In [52]:
AmFlights = (flights.where('AIRLINE', are.contained_in('AA MQ')))

Among the columns in the `flights` table is the airport where a the flight originated.  This will provide some map possibilities when we combine the airports with their latitude and longitude location.  

Let's see how many flights left from each of the airports served by American or American Eagle:

In [63]:
airport_counts = AmFlights.group('ORIGIN_AIRPORT').sort('count', descending=True)
airport_counts

ORIGIN_AIRPORT,count
DFW,187873
ORD,113871
MIA,53625
CLT,41867
LAX,32922
LGA,28274
PHX,28214
PHL,21102
JFK,19212
DCA,18621


## Sample Task: Classify airports according to number of flights

If we imagine a map where the airports are color-coded according to how many flights originate at each one, we might choose the cutoffs based on the numbers in the table above.  Let's use 5 colors and choose cutoffs at 200K, 100K, 50K, 25K, and 10K.  This code uses the `cut` method in the `pandas` library:


In [64]:
count_colors = pd.cut(airport_counts.column('count'),
     bins = [0, 10000, 25000, 50000, 100000, 200000],
     labels = ['blue', 'green', 'yellow', 'orange',
     'red'])
# Add this newly created array of colors to the Table:
airport_counts = airport_counts.with_column('colors', count_colors)

# Check out the top ten rows of the new Table:
airport_counts

ORIGIN_AIRPORT,count,colors
DFW,187873,red
ORD,113871,red
MIA,53625,orange
CLT,41867,yellow
LAX,32922,yellow
LGA,28274,yellow
PHX,28214,yellow
PHL,21102,green
JFK,19212,green
DCA,18621,green


We can add the information about each airport by `join`ing the information in the `airports` dataset:

In [65]:
airport_counts.join('ORIGIN_AIRPORT', airports, 'IATA_CODE')

ORIGIN_AIRPORT,count,colors,Airport Name,CITY,STATE,COUNTRY,lat,long
ABI,2329,blue,Abilene Regional Airport,Abilene,TX,USA,32.4113,-99.6819
ABQ,2343,blue,Albuquerque International Sunport,Albuquerque,NM,USA,35.0402,-106.609
ACT,571,blue,Waco Regional Airport,Waco,TX,USA,31.6113,-97.2305
AEX,840,blue,Alexandria International Airport,Alexandria,LA,USA,31.3274,-92.5486
ALB,346,blue,Albany International Airport,Albany,NY,USA,42.7481,-73.803
ALO,617,blue,Waterloo Regional Airport,Waterloo,IA,USA,42.5571,-92.4003
AMA,1079,blue,Rick Husband Amarillo International Airport,Amarillo,TX,USA,35.2194,-101.706
ANC,272,blue,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.1743,-149.996
ASE,534,blue,Aspen-Pitkin County Airport,Aspen,CO,USA,39.2232,-106.869
ATL,8793,blue,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.6404,-84.4269


## Sample Task: Find average AIR_TIME for each ORIGIN_AIRPORT and join lat/lon info so that we can create a map with airport location

Let's combine the idea of color-coding each airport by number of flights with the idea of taking an average per airport of some interesting variable.  Looking back at the columns in `flights`, let's try averaging the `AVG_AIRTIME` column for all American and American Eagle flights.  In the following block of code, think about what each step does with the help of the code comments:

In [66]:
# Grab only the columns we need:  AIR_TIME and ORIGIN_AIRPORT
Average_airtime = AmFlights.select('AIR_TIME', 'ORIGIN_AIRPORT')

# Now find the average of AIR_TIME for every ORIGIN_AIRPORT using group:
Average_airtime = Average_airtime.group('ORIGIN_AIRPORT', np.nanmean)

# Now join the information about the color-coding that we created above:
Average_airtime = Average_airtime.join('ORIGIN_AIRPORT', airport_counts, 'ORIGIN_AIRPORT')

# Also join the airport information that includes lat/long:
Average_airtime = Average_airtime.join('ORIGIN_AIRPORT', airports, 'IATA_CODE')

# Take a look at what we have so far:
Average_airtime

ORIGIN_AIRPORT,AIR_TIME nanmean,count,colors,Airport Name,CITY,STATE,COUNTRY,lat,long
ABI,34.0766,2329,blue,Abilene Regional Airport,Abilene,TX,USA,32.4113,-99.6819
ABQ,93.1059,2343,blue,Albuquerque International Sunport,Albuquerque,NM,USA,35.0402,-106.609
ACT,26.3121,571,blue,Waco Regional Airport,Waco,TX,USA,31.6113,-97.2305
AEX,53.0201,840,blue,Alexandria International Airport,Alexandria,LA,USA,31.3274,-92.5486
ALB,105.202,346,blue,Albany International Airport,Albany,NY,USA,42.7481,-73.803
ALO,42.6317,617,blue,Waterloo Regional Airport,Waterloo,IA,USA,42.5571,-92.4003
AMA,49.6386,1079,blue,Rick Husband Amarillo International Airport,Amarillo,TX,USA,35.2194,-101.706
ANC,320.44,272,blue,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.1743,-149.996
ASE,98.8238,534,blue,Aspen-Pitkin County Airport,Aspen,CO,USA,39.2232,-106.869
ATL,114.652,8793,blue,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.6404,-84.4269


## Sample Task: Create a map with circles

Let's use the lat/long info along with airport names as labels, counts as circle size, and the colors created earlier to indicate numbers of flights.  First, select the needed columns and then relabel:

In [67]:
Map_info = Average_airtime.select('lat', 'long', 'Airport Name', 'count', 'colors')
Map_info.relabel('Airport Name', 'labels')

lat,long,labels,count,colors
32.4113,-99.6819,Abilene Regional Airport,2329,blue
35.0402,-106.609,Albuquerque International Sunport,2343,blue
31.6113,-97.2305,Waco Regional Airport,571,blue
31.3274,-92.5486,Alexandria International Airport,840,blue
42.7481,-73.803,Albany International Airport,346,blue
42.5571,-92.4003,Waterloo Regional Airport,617,blue
35.2194,-101.706,Rick Husband Amarillo International Airport,1079,blue
61.1743,-149.996,Ted Stevens Anchorage International Airport,272,blue
39.2232,-106.869,Aspen-Pitkin County Airport,534,blue
33.6404,-84.4269,Hartsfield-Jackson Atlanta International Airport,8793,blue


Next, scale the `count` column so that we can use it as circle size on the map:

In [68]:
Map_info = Map_info.with_column('areas', Map_info.column('count')*0.03)
Map_info.show(5)

lat,long,labels,count,colors,areas
32.4113,-99.6819,Abilene Regional Airport,2329,blue,69.87
35.0402,-106.609,Albuquerque International Sunport,2343,blue,70.29
31.6113,-97.2305,Waco Regional Airport,571,blue,17.13
31.3274,-92.5486,Alexandria International Airport,840,blue,25.2
42.7481,-73.803,Albany International Airport,346,blue,10.38


Finally, we are ready to create the map using the functions seen in Section 8.5:

In [69]:
Circle.map_table(Map_info.select('lat', 'long', 'labels', 'colors', 'areas'))